In [1]:
from datasets import load_dataset
data = load_dataset("beomi/KoAlpaca-v1.1a")

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

### Instruction format for Llama 2

In [3]:
data = data.map(lambda x: {"text": f"<s>[INST] {x['instruction']}[/INST]\n\n{x['output']}</s>"})
data['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268',
 'text': '<s>[INST] 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?[/INST]\n\n양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.</s>'}

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

### Wrapping Model for PEFT

In [6]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
def print_trainable_parameter(model):
    """
    Prints the number of trainable parameters in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param = param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params/all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "down_proj",
    "gate_proj",
    "up_proj",
  ],
  lora_dropout=0.05,
  bias="none",
  task_type='CAUSAL_LM'
)
model = get_peft_model(model, config)
print_trainable_parameter(model)

trainable params: 19988480 || all params: 131072000 || trainable%: 15.25


In [9]:
!nvidia-smi

Fri Apr 26 11:32:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.65                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
|  0%   42C    P3             44W /  220W |    5347MiB /  12282MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
data['train'][:3]

{'instruction': ['양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
  '스웨터의 유래는 어디에서 시작되었나요?',
  '토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  \n\n토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?'],
 'output': ['양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
  '스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까

In [11]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir='outputs',
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=True
trainer.train()


max_steps is given, it will override any value given in num_train_epochs
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/ai/miniconda3/envs/koalpaca/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.195700
20,0.936000
30,0.938500
40,0.887900
50,0.912200
60,0.881300
70,0.893200
80,0.879600
90,0.899300
100,0.870100


TrainOutput(global_step=100, training_loss=0.9293829154968262, metrics={'train_runtime': 7192.7632, 'train_samples_per_second': 0.111, 'train_steps_per_second': 0.014, 'total_flos': 3.372909095072563e+16, 'train_loss': 0.9293829154968262, 'epoch': 0.03780718336483932})

In [12]:
model.eval()
model.config.use_cache=True

In [13]:
model.save_pretrained("llama2_alpaca_lora")

In [15]:
inputs = tokenizer("오늘 날씨는?", return_tensors="pt", return_token_type_ids=False)
outputs= model.generate(**inputs, max_length=100).to("cuda")
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

/home/ai/miniconda3/envs/koalpaca/lib/python3.11/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'오늘 날씨는? 오늘 날씨는 어떻게 될까요? 날씨 예측을 궁금합니다. \n\n\n날씨 예측은 날씨가 어떻게 될 '

In [18]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"{x}\n\n", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [19]:
gen('건강하게 살기 위한 세 가지 방법은?')

<s> 건강하게 살기 위한 세 가지 방법은?

건강하게 살기 위한 세 가지 방법은 운동, 먹는 것, 자연으로 놀러가는 것이다. 운동은 몸에 질량을 적게 줄이고 체력을 높이는 것이고, 먹는 것은 적당한 영양분을 얻는 것이며, 자연으로 놀러가는 것은 많은 것을 알 수 있고 건강한 생활 방법을 익히는 것이다. 이러한 방법들은 건강하게 살 수 있는 기본적인 방법이다. 건강하게 살기 위해서는 이러한 방법들을 적극적으로 적용해야 한다. 

운동은 몸에 질량을 적게 줄이고 체력을 높이는 것이다. 운동은 질량 관리를 위한 몇가지 방법 중 하나이다. 운동은 체력을 높이고 몸을 건강하게 유지하는 방법이다. 운동은 체력을 높이는 것


In [20]:
gen('슈카월드가 무엇인가요?')

<s> 슈카월드가 무엇인가요?

슈카월드(Sukka World)는 슈카(Sukka)라는 슈크를 뜻하는 말에서 유래한 슈크 커뮤니티의 명칭입니다. 슈크 커뮤니티는 슈크를 먹을 수 있는 모든 곳이 있는 커뮤니티를 의미하며, 슈크를 먹을 수 있는 모든 곳이 있는 곳을 의미합니다. 슈카월드는 슈크 커뮤니티를 위한 온라인 커뮤니티입니다. 이 커뮤니티에서는 슈크를 먹을 수 있는 곳, 슈크를 먹을 수 있는 매직, 슈크를 먹을 수 있는 카페, 슈크를 먹을 수 있는 음식, 슈크를 먹을 수 있는 매직 등이 있습니다. 슈카월드에서는 슈크를 먹을 수 있는 모든 곳이 있으며, 슈크를 먹을 수 있는 모든 매직이 �


In [21]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

<s> 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

주식 시장에서 안정적으로 수익을 얻기 위해서는 본질적인 기업 성과에 기초하여 적절한 주식 투자 결정을 하는 것이 중요합니다. 또한, 주식 시장에서 적절한 투자를 위해서는 금융 전문가들과 함께 투자 전략을 검토하고 적절한 주식 투자를 하는 것이 중요합니다. 이를 위해서는 본질적인 기업 성과에 기초하여 적절한 주식 투자 결정을 하는 것이 중요합니다. 또한, 주식 시장에서 적절한 투자를 위해서는 금융 전문가들과 함께 투자 전략을 검토하고 적절한 주식 투자를 하는 것이 중요합니다. 또한, 주식 시장에서 적절한 투자를 위해서는 주식 투자 전문가들과 함께 투자 전략을 검토하고 적절한 주식 투자를 하는 것이 중요합니다. 이러한 방법들은 주식 시장에서 안정적
